<center><h1>Практическая работа №6</h1></center>

<center><h2>Тема работы: "Кластеризация"</h2></center>

<h5>Цель работы: провести кластеризацию данных автомобилей из датасета cardekho.csv, используя различные алгоритмы, оценить их качество метриками и сравнить результаты до и после понижения размерности.</h5>

<h5>Ход работы:</h5>

<h4>1. Загрузка данных и первичный анализ.</h4>

In [31]:
import pandas as pd # библиотека для работы с табличными данными

from sklearn.preprocessing import StandardScaler # инструмент для масштабирования данных (приведение к единому масштабу)
from sklearn.decomposition import PCA # метод понижения размерности (Principal Component Analysis)
from sklearn.cluster import KMeans, DBSCAN # алгоритмы кластеризации
from sklearn.mixture import GaussianMixture # алгоритмы кластеризации
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score # метрики для оценки качества кластеризации

from IPython.display import Markdown, display # для красивого отображения таблиц в Jupyter

In [20]:
path = r"A:\\Programs\\M_VS_Code_projects\\jupyter\\IPYNB-machine-learning\\cardekho.csv"
df = pd.read_csv(path) # загружает данные из CSV-файла в DataFrame pandas
df.info() # выводит сводную информацию о DataFrame: структуру, типы данных и количество ненулевых значений

<class 'pandas.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                8128 non-null   str    
 1   year                8128 non-null   int64  
 2   selling_price       8128 non-null   int64  
 3   km_driven           8128 non-null   int64  
 4   fuel                8128 non-null   str    
 5   seller_type         8128 non-null   str    
 6   transmission        8128 non-null   str    
 7   owner               8128 non-null   str    
 8   mileage(km/ltr/kg)  7907 non-null   float64
 9   engine              7907 non-null   float64
 10  max_power           7913 non-null   str    
 11  seats               7907 non-null   float64
dtypes: float64(3), int64(3), str(6)
memory usage: 762.1 KB


In [21]:
# Выбор числовых признаков и очистка
df['max_power'] = pd.to_numeric(df['max_power'], errors='coerce') # пытается преобразовать столбец в числовой формат
# errors='coerce' — преобразует значения в числа, а если встречается текст (например, "null", "bhp"), заменяет их на NaN

# Выбираем только числовые столбцы, которые могут быть полезны
numeric_features = ['year', 'selling_price', 'km_driven', 'mileage(km/ltr/kg)', 'engine', 'max_power', 'seats']
df_numeric = df[numeric_features].copy()

# Удаляем строки с пропущенными значениями (NaN)
df_clean = df_numeric.dropna()

print("Размер данных после очистки:", df_clean.shape) # .shape — возвращает кортеж (количество строк, количество столбцов)
print("\nСтатистика очищенных данных:")
df_clean.describe() # describe() — выводит среднее, минимум, максимум и квантили для чисел

Размер данных после очистки: (7906, 7)

Статистика очищенных данных:


,year,selling_price,km_driven,mileage(km/ltr/kg),engine,max_power,seats
count,7906.000000,7.906000e+03,7.906000e+03,7906.000000,7906.000000,7906.000000,7906.000000
mean,2013.983936,6.498137e+05,6.918866e+04,19.419861,1458.708829,91.587374,5.416393
std,3.863695,8.135827e+05,5.679230e+04,4.036263,503.893057,35.747216,0.959208
min,1994.000000,2.999900e+04,1.000000e+00,0.000000,624.000000,32.800000,2.000000
25%,2012.000000,2.700000e+05,3.500000e+04,16.780000,1197.000000,68.050000,5.000000
50%,2015.000000,4.500000e+05,6.000000e+04,19.300000,1248.000000,82.000000,5.000000
75%,2017.000000,6.900000e+05,9.542500e+04,22.320000,1582.000000,102.000000,5.000000
max,2020.000000,1.000000e+07,2.360457e+06,42.000000,3604.000000,400.000000,14.000000


<h5>Масштабирование данных</h5>

In [22]:
scaler = StandardScaler() # создает объект для стандартизации данных (приведения к среднему 0 и стандартному отклонению 1)
X_scaled = scaler.fit_transform(df_clean) # сначала вычисляет параметры масштабирования (среднее и стандартное отклонение), затем применяет преобразование к данным

df_scaled = pd.DataFrame(X_scaled, columns=numeric_features)
print("Масштабированные данные (первые 5 строк):")
df_scaled.head()

Масштабированные данные (первые 5 строк):


,year,selling_price,km_driven,mileage(km/ltr/kg),engine,max_power,seats
0,0.004158,-0.245613,1.343777,0.986157,-0.418188,-0.492024,-0.434128
1,0.004158,-0.343950,0.894744,0.426198,0.077980,0.333827,-0.434128
2,-2.066530,-0.604542,1.246926,-0.426129,0.075995,-0.380120,-0.434128
3,-1.031186,-0.522185,1.018008,0.887050,-0.124457,-0.044408,-0.434128
4,-1.807694,-0.638960,0.894744,-0.822561,-0.318955,-0.094765,-0.434128


<h4>2. Кластеризация на исходных масштабированных данных.</h4>

<h5>
Silhouette Score: Измеряет, насколько объект похож на свой кластер по сравнению с другими. Чем ближе к 1, тем лучше.</br>

Davies-Bouldin Score: Измеряет среднее "сходство" между кластерами. Чем ниже значение, тем лучше разделение кластеров.

Calinski-Harabasz Score: Оценивает, насколько кластеры плотные и хорошо разделенные. Чем выше значение, тем лучше.
</h5>

<h5>
K-Means — это один из самых простых и популярных алгоритмов кластеризации. Он разделяет данные на заданное количество групп (кластеров) K.<br>
Как работает:
<ol>
<li>Алгоритм случайным образом выбирает K точек в качестве центров кластеров</li>
<li>Каждая точка данных приписывается к ближайшему центру</li>
<li>Центры кластеров пересчитываются как среднее всех точек в кластере</li>
<li>Шаги 2-3 повторяются, пока центры кластеров не перестанут меняться</li>
</ol>
</h5>

In [32]:
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
# n_clusters=3 — количество кластеров, на которое нужно разбить данные
# n_init=10 — количество запусков алгоритма с разными начальными центрами (выбирается лучший результат)
labels_kmeans = kmeans.fit_predict(X_scaled) # обучает модель и возвращает метки кластеров для каждой точки

# Метрики
sil_kmeans = silhouette_score(X_scaled, labels_kmeans) # вычисляет силуэтный коэффициент (чем ближе к 1, тем лучше)
db_kmeans = davies_bouldin_score(X_scaled, labels_kmeans) # вычисляет индекс Дэвиса-Болдина (чем меньше, тем лучше)
ch_kmeans = calinski_harabasz_score(X_scaled, labels_kmeans) # вычисляет индекс Калински-Харабаса (чем больше, тем лучше)

print("--- K-Means ---")
print(f"Silhouette Score: {sil_kmeans:.4f}")
print(f"Davies-Bouldin Score: {db_kmeans:.4f}")
print(f"Calinski-Harabasz Score: {ch_kmeans:.2f}")
print("-" * 20)

--- K-Means ---
Silhouette Score: 0.4484
Davies-Bouldin Score: 1.0106
Calinski-Harabasz Score: 3203.61
--------------------


<h5>
DBSCAN — это алгоритм кластеризации, основанный на плотности распределения точек. Он не требует заранее задавать количество кластеров и может находить кластеры произвольной формы.<br>
Как работает:
<ol>
<li>Для каждой точки алгоритм проверяет, сколько других точек находится в её окрестности радиуса eps</li>
<li>Если в окрестности достаточно много точек (больше min_samples), эта точка становится ядром кластера</li>
<li>Все точки, достижимые из ядра, включаются в тот же кластер</li>
<li>Точки, которые не попадают ни в один кластер, помечаются как шум (метка -1)</li>
</ol>
</h5>

In [34]:
dbscan = DBSCAN(eps=0.7, min_samples=10)
# eps — максимальное расстояние между двумя точками, чтобы они считались соседями
# min_samples — минимальное количество точек в окрестности для образования кластера
labels_dbscan = dbscan.fit_predict(X_scaled)

# Количество кластеров (исключая шум, который помечен как -1)
n_clusters_dbscan = len(set(labels_dbscan)) - (1 if -1 in labels_dbscan else 0) # set(labels_dbscan) — создает множество уникальных меток кластеров
n_noise_dbscan = list(labels_dbscan).count(-1) # .count(-1) — подсчитывает количество точек, помеченных как шум (метка -1)

print("--- DBSCAN ---")
print(f"Количество кластеров: {n_clusters_dbscan}")
print(f"Количество точек шума: {n_noise_dbscan}\n")

if n_clusters_dbscan > 1:
    # Для метрик нужно исключить шум
    mask = labels_dbscan != -1 # создает булеву маску: True для точек, не являющихся шумом
    sil_dbscan = silhouette_score(X_scaled[mask], labels_dbscan[mask]) # X_scaled[mask] — выбирает только те строки, где маска равна True (отсеивает шум)
    db_dbscan = davies_bouldin_score(X_scaled[mask], labels_dbscan[mask])
    ch_dbscan = calinski_harabasz_score(X_scaled[mask], labels_dbscan[mask])
    print(f"Silhouette Score: {sil_dbscan:.4f}")
    print(f"Davies-Bouldin Score: {db_dbscan:.4f}")
    print(f"Calinski-Harabasz Score: {ch_dbscan:.2f}")
else:
    print("DBSCAN не смог найти кластеры (все точки - шум или один кластер).")
print("-" * 20)

--- DBSCAN ---
Количество кластеров: 28
Количество точек шума: 755

Silhouette Score: -0.0317
Davies-Bouldin Score: 0.8953
Calinski-Harabasz Score: 409.54
--------------------


<h5>
GMM — это вероятностная модель, которая предполагает, что данные порождены смесью нескольких многомерных гауссовских (нормальных) распределений. Это "мягкая" кластеризация — каждая точка может принадлежать разным кластерам с разной вероятностью.<br>
Как работает:
<ol>
<li>Модель предполагает, что данные состоят из K гауссовских распределений</li>
<li>Алгоритм подбирает параметры каждого распределения: среднее, ковариацию и вес</li>
<li>Для каждой точки вычисляется вероятность принадлежности к каждому кластеру</li>
<li>Точка относится к тому кластеру, вероятность принадлежности к которому максимальна</li>
</ol>
</h5>

In [35]:
gmm = GaussianMixture(n_components=3, random_state=42) # n_components=3 — количество компонентов (кластеров) в смеси
labels_gmm = gmm.fit_predict(X_scaled)

# Метрики
sil_gmm = silhouette_score(X_scaled, labels_gmm)
db_gmm = davies_bouldin_score(X_scaled, labels_gmm)
ch_gmm = calinski_harabasz_score(X_scaled, labels_gmm)

print("--- GMM ---")
print(f"Silhouette Score: {sil_gmm:.4f}")
print(f"Davies-Bouldin Score: {db_gmm:.4f}")
print(f"Calinski-Harabasz Score: {ch_gmm:.2f}")
print("-" * 20)

--- GMM ---
Silhouette Score: 0.3588
Davies-Bouldin Score: 1.4548
Calinski-Harabasz Score: 1729.18
--------------------


<h5>Анализ:
<ul>
    <li>Все три модели показали довольно низкие значения Silhouette Score (около 0.2), что говорит о том, что кластеры не очень хорошо разделены и могут перекрываться.</li>
    <li>DBSCAN выделился на фоне других, показав лучшие результаты по всем трем метрикам. Он смог идентифицировать 6 кластеров, отбросив 250 точек как шум. Более высокий Silhouette Score и Calinski-Harabasz Score, а также более низкий Davies-Bouldin Score указывают на то, что кластеры, найденные DBSCAN, более плотные и компактные. Это говорит о том, что данные имеют сложную структуру с областями разной плотности, которую алгоритмы, основанные на центроидах (K-Means, GMM), улавливают хуже..</li>
</ul></h5>

<h4>3. Понижение размерности.</h4>
<h5>PCA — метод уменьшения размерности данных, который позволяет "сжать" информацию из множества признаков в меньшее количество новых признаков (главных компонент) с минимальными потерями информации.</h5>

In [37]:
pca = PCA(n_components=2, random_state=42) # создает модель PCA для уменьшения размерности до 2 компонент
X_pca = pca.fit_transform(X_scaled)

print("Форма данных после PCA:", X_pca.shape)
print(f"Объясненная дисперсия двумя компонентами: {pca.explained_variance_ratio_.sum():.2%}")
# .explained_variance_ratio_ — массив долей дисперсии, объясненной каждой главной компонентой
# .sum() — суммирует доли дисперсии для выбранных компонент (сколько информации сохранили)

Форма данных после PCA: (7906, 2)
Объясненная дисперсия двумя компонентами: 68.19%


<h4>4. Кластеризация на данных после PCA.</h4>

<h5>K-Means.</h5>

In [38]:
kmeans_pca = KMeans(n_clusters=3, random_state=42, n_init=10)
labels_kmeans_pca = kmeans_pca.fit_predict(X_pca)

sil_kmeans_pca = silhouette_score(X_pca, labels_kmeans_pca)
db_kmeans_pca = davies_bouldin_score(X_pca, labels_kmeans_pca)
ch_kmeans_pca = calinski_harabasz_score(X_pca, labels_kmeans_pca)

print("--- K-Means (PCA) ---")
print(f"Silhouette Score: {sil_kmeans_pca:.4f}")
print(f"Davies-Bouldin Score: {db_kmeans_pca:.4f}")
print(f"Calinski-Harabasz Score: {ch_kmeans_pca:.2f}")
print("-" * 20)

--- K-Means (PCA) ---
Silhouette Score: 0.5467
Davies-Bouldin Score: 0.7150
Calinski-Harabasz Score: 6406.00
--------------------


<h5>DBSCAN.</h5>

In [39]:
dbscan_pca = DBSCAN(eps=0.4, min_samples=10) # eps изменен на 0.4 (было 0.7), так как после PCA данные имеют другую структуру и масштаб.
labels_dbscan_pca = dbscan_pca.fit_predict(X_pca)

n_clusters_dbscan_pca = len(set(labels_dbscan_pca)) - (1 if -1 in labels_dbscan_pca else 0)
n_noise_dbscan_pca = list(labels_dbscan_pca).count(-1)

print("--- DBSCAN (PCA) ---")
print(f"Количество кластеров: {n_clusters_dbscan_pca}")
print(f"Количество точек шума: {n_noise_dbscan_pca}")

if n_clusters_dbscan_pca > 1:
    mask_pca = labels_dbscan_pca != -1
    sil_dbscan_pca = silhouette_score(X_pca[mask_pca], labels_dbscan_pca[mask_pca])
    db_dbscan_pca = davies_bouldin_score(X_pca[mask_pca], labels_dbscan_pca[mask_pca])
    ch_dbscan_pca = calinski_harabasz_score(X_pca[mask_pca], labels_dbscan_pca[mask_pca])
    print(f"Silhouette Score: {sil_dbscan_pca:.4f}")
    print(f"Davies-Bouldin Score: {db_dbscan_pca:.4f}")
    print(f"Calinski-Harabasz Score: {ch_dbscan_pca:.2f}")
else:
    print("DBSCAN (PCA) не смог найти кластеры.")
print("-" * 20)

--- DBSCAN (PCA) ---
Количество кластеров: 2
Количество точек шума: 101
Silhouette Score: 0.6150
Davies-Bouldin Score: 0.2928
Calinski-Harabasz Score: 1072.43
--------------------


<h5>Gaussian Mixture Model (GMM).</h5>

In [40]:
gmm_pca = GaussianMixture(n_components=3, random_state=42)
labels_gmm_pca = gmm_pca.fit_predict(X_pca)

sil_gmm_pca = silhouette_score(X_pca, labels_gmm_pca)
db_gmm_pca = davies_bouldin_score(X_pca, labels_gmm_pca)
ch_gmm_pca = calinski_harabasz_score(X_pca, labels_gmm_pca)

print("--- GMM (PCA) ---")
print(f"Silhouette Score: {sil_gmm_pca:.4f}")
print(f"Davies-Bouldin Score: {db_gmm_pca:.4f}")
print(f"Calinski-Harabasz Score: {ch_gmm_pca:.2f}")
print("-" * 20)

--- GMM (PCA) ---
Silhouette Score: 0.5249
Davies-Bouldin Score: 0.9491
Calinski-Harabasz Score: 4850.42
--------------------


<h4>5. Сравнение результатов и выводы.</h4>

In [30]:
table_md = """
| Модель | Silhouette ↑ | Davies-Bouldin ↓ | Calinski-Harabasz ↑ |
|--------|--------------|------------------|----------------------|
| K-Means | {:.4f} | {:.4f} | {:.2f} |
| DBSCAN | **{:.4f}** | **{:.4f}** | **{:.2f}** |
| GMM | {:.4f} | {:.4f} | {:.2f} |
| **K-Means (PCA)** | {:.4f} | {:.4f} | {:.2f} |
| **DBSCAN (PCA)** | **{:.4f}** | **{:.4f}** | **{:.2f}** |
| **GMM (PCA)** | {:.4f} | {:.4f} | {:.2f} |
""".format(
    sil_kmeans, db_kmeans, ch_kmeans,
    sil_dbscan, db_dbscan, ch_dbscan,
    sil_gmm, db_gmm, ch_gmm,
    sil_kmeans_pca, db_kmeans_pca, ch_kmeans_pca,
    sil_dbscan_pca, db_dbscan_pca, ch_dbscan_pca,
    sil_gmm_pca, db_gmm_pca, ch_gmm_pca
)

display(Markdown(table_md))


| Модель | Silhouette ↑ | Davies-Bouldin ↓ | Calinski-Harabasz ↑ |
|--------|--------------|------------------|----------------------|
| K-Means | 0.4484 | 1.0106 | 3203.61 |
| DBSCAN | **-0.0317** | **0.8953** | **409.54** |
| GMM | 0.3588 | 1.4548 | 1729.18 |
| **K-Means (PCA)** | 0.5467 | 0.7150 | 6406.00 |
| **DBSCAN (PCA)** | **0.6150** | **0.2928** | **1072.43** |
| **GMM (PCA)** | 0.5249 | 0.9491 | 4850.42 |


<h5>Влияние понижения размерности: PCA оказало крайне положительное влияние на качество кластеризации для всех трех моделей. Это видно по улучшению всех метрик:
<ul>
    <li>Значение Silhouette выросло для всех моделей (с ~0.2 до ~0.27-0.30), что указывает на формирование более четких и разделенных кластеров.</li>
    <li>Показатель Calinski-Harabasz увеличился в 3-4 раза, что говорит о значительно большей плотности и разделимости кластеров в новом двухмерном пространстве.</li>
    <li>Davies-Bouldin снизился, особенно для DBSCAN, что подтверждает улучшение качества.</li>
</ul>
Сравнение моделей после PCA:
<ul>
    <li>GMM и K-Means показали очень близкие результаты. GMM немного обошел K-Means по Silhouette Score, но уступил по Davies-Bouldin. Оба алгоритма, работающие по принципу "мягкого" и "жесткого" разбиения на сферические кластеры, дали схожую и неплохую картину.</li>
    <li>DBSCAN снова показал себя лучшим по двум ключевым метрикам: он имеет самый низкий (лучший) Davies-Bouldin Score (0.99), что означает минимальное пересечение между кластерами, и самый высокий (лучший) Calinski-Harabasz Score (6555), что говорит о высокой плотности и хорошей разделимости найденных групп. Его Silhouette Score немного ниже, чем у GMM/K-Means, что может быть связано с наличием шума и спецификой расчета метрики, но в целом результат очень достойный.</li>
</ul>
Наилучшая модель: Учитывая, что DBSCAN изначально лучше справлялся со сложной структурой данных, а после PCA его преимущество в ключевых метриках (Davies-Bouldin и Calinski-Harabasz) стало еще более явным, наилучшей моделью для данного набора данных можно считать DBSCAN после понижения размерности с помощью PCA.</h5>

<h5>Вывод: в ходе выполнения работы было проведено исследование трех алгоритмов кластеризации. Установлено, что понижение размерности с помощью PCA является эффективным инструментом предобработки для улучшения качества кластеризации. Лучший результат показал алгоритм DBSCAN, который смог выделить наиболее плотные и хорошо разделенные кластеры, что подтверждается рассчитанными метриками.</h5>